In [1]:
import os
import time
import json
import pprint
import random
import numpy as np
from tqdm import tqdm, trange
from collections import defaultdict

In [2]:
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader



In [5]:
from torch.utils.tensorboard import SummaryWriter

from cg_detr.config import BaseOptions
from cg_detr.start_end_dataset import \
    StartEndDataset, start_end_collate, prepare_batch_inputs
from cg_detr.inference import eval_epoch, start_inference, setup_model
from utils.basic_utils import AverageMeter, dict_to_markdown
from utils.model_utils import count_parameters

OSError: [WinError 127] The specified procedure could not be found

In [4]:
# Initialize video feature info
v_feat_dirs = []
v_feat_dim = 0

In [5]:
if "slowfast" in v_feat_types:
    v_feat_dirs.append(f"{feat_root}/slowfast_features")
    v_feat_dim += 2304

In [6]:
if "clip" in v_feat_types:
    v_feat_dirs.append(f"{feat_root}/clip_features")
    v_feat_dim += 512

In [7]:
# Text feature
if t_feat_type == "clip":
    t_feat_dir = f"{feat_root}/clip_text_features/"
    t_feat_dim = 512
else:
    raise ValueError("Wrong arg for t_feat_type.")

In [8]:
# Step 2: Set training hyperparameters

bsz = 32
enc_layers = 3
dec_layers = 3
t2v_layers = 2
moment_layers = 1
dummy_layers = 2
sent_layers = 1
max_v_l = 75
max_q_l = 32

In [9]:
# Step 3: Construct command and run training

import subprocess

command = [
    "python", "cg_detr/train.py",
    "--dset_name", dset_name,
    "--ctx_mode", ctx_mode,
    "--train_path", train_path,
    "--eval_path", eval_path,
    "--eval_split_name", eval_split_name,
    "--v_feat_dim", str(v_feat_dim),
    "--t_feat_dir", t_feat_dir,
    "--t_feat_dim", str(t_feat_dim),
    "--bsz", str(bsz),
    "--results_root", results_root,
    "--exp_id", exp_id,
    "--enc_layers", str(enc_layers),
    "--dec_layers", str(dec_layers),
    "--t2v_layers", str(t2v_layers),
    "--moment_layers", str(moment_layers),
    "--dummy_layers", str(dummy_layers),
    "--sent_layers", str(sent_layers),
    "--max_v_l", str(max_v_l),
    "--max_q_l", str(max_q_l)
]

In [10]:
# Add multiple v_feat_dirs
for v_dir in v_feat_dirs:
    command.extend(["--v_feat_dirs", v_dir])

# Optional: print the full command for debugging
print("Running command:\n", " ".join(command))

Running command:
 python cg_detr/train.py --dset_name hl --ctx_mode video_tef --train_path data/highlight_train_release.jsonl --eval_path data/highlight_val_release.jsonl --eval_split_name val --v_feat_dim 2816 --t_feat_dir ../features/qvhighlight/clip_text_features/ --t_feat_dim 512 --bsz 32 --results_root results --exp_id exp --enc_layers 3 --dec_layers 3 --t2v_layers 2 --moment_layers 1 --dummy_layers 2 --sent_layers 1 --max_v_l 75 --max_q_l 32 --v_feat_dirs ../features/qvhighlight/slowfast_features --v_feat_dirs ../features/qvhighlight/clip_features


In [12]:
!pip install numpy

In [13]:
# Step 4: Run the training script

result = subprocess.run(command, capture_output=True, text=True)

print("STDOUT:\n", result.stdout)
print("STDERR:\n", result.stderr)

STDOUT:
 
STDERR:
 Traceback (most recent call last):
  File "f:\CG-Detr\CGDETR\cg_detr\train.py", line 6, in <module>
    import numpy as np
ModuleNotFoundError: No module named 'numpy'



In [14]:
import sys
print(sys.executable)

f:\CG-Detr\CGDETR\venv\Scripts\python.exe
